<h1><center>Data Science for Covid19</center></h1>
<center>Intelligent Systems For Bioinformatics</center>

For this workflow it will be necessary import the following packages:

In [91]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore') 
import pickle

seed = 123
np.random.seed(seed)

The dataset "Covid_Dataset.csv" contains several simptoms and features associated to Covid-19 that were evaluated in 5434 different patients along with the testing result for this disease.

In [92]:
dataset = pd.read_csv("Covid_Dataset.csv")
dataset

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,Yes,Yes,Yes,Yes,Yes,No,No,No,No,Yes,...,Yes,Yes,No,Yes,No,Yes,Yes,No,No,Yes
1,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,No,No,...,Yes,No,No,No,Yes,Yes,No,No,No,Yes
2,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,...,Yes,Yes,Yes,No,No,No,No,No,No,Yes
3,Yes,Yes,Yes,No,No,Yes,No,No,Yes,Yes,...,No,No,Yes,No,Yes,Yes,No,No,No,Yes
4,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,...,No,Yes,No,Yes,No,Yes,No,No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,Yes,Yes,No,Yes,Yes,Yes,Yes,No,No,No,...,Yes,Yes,No,No,No,No,No,No,No,Yes
5430,Yes,Yes,Yes,No,Yes,Yes,No,Yes,No,Yes,...,Yes,No,No,No,No,No,No,No,No,Yes
5431,Yes,Yes,Yes,No,No,No,No,No,Yes,No,...,No,No,No,No,No,No,No,No,No,No
5432,Yes,Yes,Yes,No,Yes,No,No,Yes,Yes,No,...,No,No,No,No,No,No,No,No,No,No


In [93]:
print("Number of samples: " + str(dataset.shape[0]))
print("Number of features: " + str(dataset.shape[1]))
print("Number of NAs: " + str(sum(dataset.isna().sum())))

Number of samples: 5434
Number of features: 21
Number of NAs: 0


The features evaluated for all the patients are:

In [94]:
for col in dataset.columns:
    print(col)

Breathing Problem
Fever
Dry Cough
Sore throat
Running Nose
Asthma
Chronic Lung Disease
Headache
Heart Disease
Diabetes
Hyper Tension
Fatigue 
Gastrointestinal 
Abroad travel
Contact with COVID Patient
Attended Large Gathering
Visited Public Exposed Places
Family working in Public Exposed Places
Wearing Masks
Sanitization from Market
COVID-19


Check for NAs on dataset 

In [95]:
dataset.isna().sum()

Breathing Problem                          0
Fever                                      0
Dry Cough                                  0
Sore throat                                0
Running Nose                               0
Asthma                                     0
Chronic Lung Disease                       0
Headache                                   0
Heart Disease                              0
Diabetes                                   0
Hyper Tension                              0
Fatigue                                    0
Gastrointestinal                           0
Abroad travel                              0
Contact with COVID Patient                 0
Attended Large Gathering                   0
Visited Public Exposed Places              0
Family working in Public Exposed Places    0
Wearing Masks                              0
Sanitization from Market                   0
COVID-19                                   0
dtype: int64

Once the variables were all "Yes" or "No", we transform the variables into binary using LabelEncoder from ScikitLearn.

In [96]:
dataset = dataset.apply(preprocessing.LabelEncoder().fit_transform)
dataset

,Breathing Problem,Fever,Dry Cough,Sore throat,Running Nose,Asthma,Chronic Lung Disease,Headache,Heart Disease,Diabetes,...,Fatigue,Gastrointestinal,Abroad travel,Contact with COVID Patient,Attended Large Gathering,Visited Public Exposed Places,Family working in Public Exposed Places,Wearing Masks,Sanitization from Market,COVID-19
0,1,1,1,1,1,0,0,0,0,1,...,1,1,0,1,0,1,1,0,0,1
1,1,1,1,1,0,1,1,1,0,0,...,1,0,0,0,1,1,0,0,0,1
2,1,1,1,1,1,1,1,1,0,1,...,1,1,1,0,0,0,0,0,0,1
3,1,1,1,0,0,1,0,0,1,1,...,0,0,1,0,1,1,0,0,0,1
4,1,1,1,1,1,0,1,1,1,1,...,0,1,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,1,1,0,1,1,1,1,0,0,0,...,1,1,0,0,0,0,0,0,0,1
5430,1,1,1,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,0,0,1
5431,1,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5432,1,1,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


We then proceed to separate the input variables (X) from the output variable (y)

In [97]:
X = dataset.drop('COVID-19', axis=1)
y = dataset['COVID-19'].values

The construction of training and test sets was executed

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2173, 20)
(3261, 20)
(2173,)
(3261,)


The complexity of the model increases with the increase in the number of features, which can lead to the occurrence of overfitting. This happens when the model fits too well to the training dataset, but then cannot generalize to new examples that the model has never seen during training. In order to avoid this problem we can choose to reduce the number of features, in this case we used the SelectKBest class that selects the k features with the best score based on univariate statistical tests.

In [99]:
selector = SelectKBest(f_regression)

### Random Forest 

For first approach is used the Random Forest algorithm where the scikit Learn RandomForestClassifier class was used as estimator

In [100]:
estimator =  RandomForestClassifier()
estimator

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

The Pipeline class is used to chain a series of processing that is done to the data, in this case, the selection of the best k features using the selector estimator defined above and the model estimator itself.

In [101]:
pipeline = Pipeline([('selector', selector),('estimator',estimator)])

Then several hyperparameters were tested using the RandomizedSearchCV optimization method, which tests some random combinations of the search grid values, evaluating performance through cross-validation (3 folds). After defining the hyperparameter grid and the respective values to be tested, the best ones were chosen for the future construction of the model.

In [102]:
params_to_test = {'estimator__n_estimators':[100, 200, 500, 700, 1000],'selector__k':[5, 10, 15, 'all']}
        
randomized_search = RandomizedSearchCV(pipeline,params_to_test, cv=3)
randomized_search.fit(X,y)

rs_results = pd.DataFrame.from_dict(data=randomized_search.cv_results_)
rs_results.to_csv('rs_results(RF).csv',index=False)
rs_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selector__k,param_estimator__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.566341,0.052204,0.135728,0.002407,all,700,"{'selector__k': 'all', 'estimator__n_estimator...",0.937086,0.910547,0.902816,0.916816,0.014676,5
1,1.485357,0.017372,0.134350,0.001399,15,700,"{'selector__k': 15, 'estimator__n_estimators':...",0.937086,0.920486,0.902816,0.920129,0.013993,3
2,2.162813,0.100079,0.195199,0.006366,15,1000,"{'selector__k': 15, 'estimator__n_estimators':...",0.937086,0.904473,0.902816,0.914792,0.015779,8
3,2.023622,0.022945,0.193323,0.009732,10,1000,"{'selector__k': 10, 'estimator__n_estimators':...",0.955298,0.915516,0.895638,0.922151,0.024804,1
4,0.921435,0.005541,0.088593,0.001254,5,500,"{'selector__k': 5, 'estimator__n_estimators': ...",0.956954,0.887355,0.717283,0.853864,0.100670,10
5,0.227950,0.004097,0.020584,0.000253,all,100,"{'selector__k': 'all', 'estimator__n_estimator...",0.932119,0.925456,0.902816,0.920130,0.012542,2
6,0.222001,0.003871,0.020732,0.000270,15,100,"{'selector__k': 15, 'estimator__n_estimators':...",0.937086,0.908338,0.911651,0.919025,0.012843,4
7,0.467508,0.037638,0.041148,0.001957,15,200,"{'selector__k': 15, 'estimator__n_estimators':...",0.937086,0.904473,0.902816,0.914792,0.015779,8
8,0.447959,0.005008,0.039708,0.000773,all,200,"{'selector__k': 'all', 'estimator__n_estimator...",0.932119,0.914412,0.902816,0.916449,0.012049,6
9,1.044513,0.011026,0.096139,0.001229,15,500,"{'selector__k': 15, 'estimator__n_estimators':...",0.937086,0.908338,0.902816,0.916080,0.015024,7


The best hyperparameters were:

In [103]:
randomized_search.best_params_

{'selector__k': 10, 'estimator__n_estimators': 1000}

Building the model using the best hyperparameters

In [104]:
best_pipeline = randomized_search.best_estimator_
best_pipeline

Pipeline(memory=None,
         steps=[('selector',
                 SelectKBest(k=10,
                             score_func=<function f_regression at 0x7f96b0866cb0>)),
                ('estimator',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=1000, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=Fa

Now cross validation is used to evaluate the model with the hyperparameters already optimized. In this step different metrics can be used, such as the accuracy, recall, precision and area under the curve (AUC).

In [105]:
scoring_metrics = ['accuracy', 'recall', 'precision', 'roc_auc']

cv_scores = cross_validate(best_pipeline, X, y, scoring=scoring_metrics, cv=5, return_train_score=False)

for key, value in cv_scores.items():
    if key != "fit_time" and key != "score_time":
        print("Metrics: %s" % key)
        print("Results for each fold: %s" % value) 
        print("Mean for all folds: %s" % np.mean(value))
        print("Standard deviation: %s" % np.std(value))

Metrics: test_accuracy
Results for each fold: [0.93100276 0.99172033 0.89604416 0.78656854 0.8664825 ]
Mean for all folds: 0.8943636582345178
Standard deviation: 0.06814795147715773
Metrics: test_recall
Results for each fold: [0.99201824 1.         0.9281642  0.73744292 0.92694064]
Mean for all folds: 0.916913200356133
Standard deviation: 0.09484819341832654
Metrics: test_precision
Results for each fold: [0.92750533 0.98984199 0.94212963 0.99691358 0.90929451]
Mean for all folds: 0.953137007940174
Standard deviation: 0.034536801734427046
Metrics: test_roc_auc
Results for each fold: [0.98633871 1.         0.94825433 0.94980956 0.86593281]
Mean for all folds: 0.9500670822857644
Standard deviation: 0.04667087972751376


The model is trained again with the complete dataset before being used to make predictions for test datasets.

In [106]:
modelRF = best_pipeline.fit(X_train,y_train)

In the case of the Random Forest algorithm it is possible to obtain information about the importance of each feature

In [107]:
ln = X.shape
names = X.columns.tolist()
feature_importances = sorted(zip(map(lambda x: round(x, 4), modelRF.named_steps['estimator'].feature_importances_),names), reverse=True)

feature_importances = pd.DataFrame.from_dict(data=feature_importances)
feature_importances.to_csv('feature_importances(RF).csv',index=False)

feature_importances.head()

,0,1
0,0.1828,Sore throat
1,0.1629,Breathing Problem
2,0.1567,Dry Cough
3,0.1534,Asthma
4,0.1164,Headache


Finally, we predicted the final results for the COVID-19 variable using the test dataset splited above and evaluate the performance of this results.

In [108]:
y_test_pred = modelRF.predict(X_test)
y_test_pred

array([1, 1, 1, ..., 1, 1, 1])

In [109]:
score = accuracy_score(y_test, y_test_pred)
score

0.9763876111622202

The model can be saved for future use:

In [110]:
pickle.dump(modelRF, open('modelRF.pkl', 'wb'))

### Support Vector Machine

For this approach is used the Support Vector Machine algorithm where the scikit Learn SVR class was used as estimator.

In [111]:
estimator =  SVC(kernel = 'rbf')
estimator

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

The Pipeline class is used to chain a series of processing that is done to the data, in this case, the selection of the best k features using the selector estimator defined above and the model estimator itself.

In [112]:
pipeline = Pipeline([('selector', selector),('estimator', estimator)])

Then several hyperparameters were tested using the RandomizedSearchCV optimization method, which tests some random combinations of the search grid values, evaluating performance through cross-validation (3 folds). After defining the hyperparameter grid and the respective values to be tested, the best ones were chosen for the future construction of the model.

In [113]:
params_to_test = {'estimator__C':[0.1, 1, 10],'selector__k':[5, 10, 15, 'all']}
        
randomized_search = RandomizedSearchCV(pipeline,params_to_test, cv=3)
randomized_search.fit(X,y)

rs_results = pd.DataFrame.from_dict(data=randomized_search.cv_results_)
rs_results.to_csv('rs_results(SVM).csv',index=False)
rs_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_selector__k,param_estimator__C,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.137369,0.042706,0.045662,0.007078,all,0.1,"{'selector__k': 'all', 'estimator__C': 0.1}",0.974614,0.916621,0.894533,0.928589,0.033770,4
1,0.054053,0.016965,0.019494,0.005499,5,0.1,"{'selector__k': 5, 'estimator__C': 0.1}",0.956954,0.887355,0.805632,0.883314,0.061843,9
2,0.053031,0.011543,0.018991,0.004125,all,1,"{'selector__k': 'all', 'estimator__C': 1}",0.970199,0.912755,0.905025,0.929326,0.029073,3
3,0.080046,0.017205,0.028457,0.006051,10,0.1,"{'selector__k': 10, 'estimator__C': 0.1}",0.971854,0.914412,0.888459,0.924909,0.034845,6
4,0.038649,0.006676,0.012582,0.002289,15,10,"{'selector__k': 15, 'estimator__C': 10}",0.963576,0.904473,0.901160,0.923069,0.028674,7
5,0.050268,0.010526,0.020569,0.008240,15,1,"{'selector__k': 15, 'estimator__C': 1}",0.958609,0.904473,0.899503,0.920862,0.026769,8
6,0.035388,0.012624,0.012508,0.004811,5,10,"{'selector__k': 5, 'estimator__C': 10}",0.956954,0.887355,0.698509,0.847606,0.109189,10
7,0.043547,0.006911,0.014001,0.002165,all,10,"{'selector__k': 'all', 'estimator__C': 10}",0.970199,0.912755,0.909994,0.930983,0.027753,2
8,0.041610,0.011590,0.014295,0.003876,10,1,"{'selector__k': 10, 'estimator__C': 1}",0.965232,0.939260,0.900055,0.934849,0.026790,1
9,0.106942,0.018810,0.039312,0.006962,15,0.1,"{'selector__k': 15, 'estimator__C': 0.1}",0.970199,0.924351,0.889564,0.928038,0.033022,5


The best hyperparameters were:

In [114]:
randomized_search.best_params_

{'selector__k': 10, 'estimator__C': 1}

Building the model using the best hyperparameters

In [115]:
best_pipeline = randomized_search.best_estimator_
best_pipeline

Pipeline(memory=None,
         steps=[('selector',
                 SelectKBest(k=10,
                             score_func=<function f_regression at 0x7f96b0866cb0>)),
                ('estimator',
                 SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

Now cross validation is used to evaluate the model with the hyperparameters already optimized. In this step different metrics can be used, such as the accuracy, recall, precision and area under the curve (AUC).

In [116]:
scoring_metrics = ['accuracy', 'recall', 'precision', 'roc_auc']

cv_scores = cross_validate(best_pipeline, X, y, scoring=scoring_metrics, cv=5, return_train_score=False)

for key, value in cv_scores.items():
    if key != "fit_time" and key != "score_time":
        print("Metrics: %s" % key)
        print("Results for each fold: %s" % value) 
        print("Mean for all folds: %s" % np.mean(value))
        print("Standard deviation: %s" % np.std(value))

Metrics: test_accuracy
Results for each fold: [0.96964121 0.98344066 0.88684453 0.83716651 0.90976059]
Mean for all folds: 0.9173707011203897
Standard deviation: 0.05388862445223599
Metrics: test_recall
Results for each fold: [0.99201824 1.         0.9281642  0.80022831 0.97374429]
Mean for all folds: 0.9388310085753112
Standard deviation: 0.07363960035548202
Metrics: test_precision
Results for each fold: [0.97098214 0.97988827 0.93135011 0.99715505 0.91918103]
Mean for all folds: 0.9597113219398862
Standard deviation: 0.029608227273439407
Metrics: test_roc_auc
Results for each fold: [0.99776294 1.         0.92227833 0.9704089  0.85422918]
Mean for all folds: 0.9489358699011105
Standard deviation: 0.055013999066512644


The model is trained again with the complete dataset before being used to make predictions for test datasets.

In [117]:
modelSVM = best_pipeline.fit(X_train,y_train)

Finally, we predicted the final results for the COVID-19 variable using the test dataset splited above and evaluate the performance of this results.

In [118]:
y_test_pred = modelSVM.predict(X_test)
y_test_pred

array([1, 1, 1, ..., 1, 1, 1])

In [119]:
score = accuracy_score(y_test, y_test_pred)
score

0.9708678319533885

The model can be saved for future use:

In [120]:
pickle.dump(modelSVM, open('modelSVM.pkl', 'wb'))